In [54]:
%%file finitediff.c

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <sys/stat.h>
#include <sys/types.h>
#include <direct.h>
#include <errno.h>

//Constants
#define L 1.0         // Box length
#define a 0.1         // Initial Gaussian width
#define c 1.0         // Speed of sound
#define Nx 100        // Number of spatial points
#define dx (L / (Nx - 1))   // Spatial step size (fixed)

//Initialize the wave with Gaussian profile and zero initial velocity
void initialize_wave(double *f, double *v, double *x) {
    for (int i = 0; i < Nx; i++) {
        x[i] = -L / 2 + i * dx;
        f[i] = exp(-x[i] * x[i] / (a * a));  // Gaussian initial condition
        v[i] = 0.0;
    }
}

//Compute second spatial derivative using fourth-order finite differencing
void compute_spatial_derivatives(double *f, double *d2f_dx2) {
    
    for (int i = 2; i < Nx - 2; i++) {
        d2f_dx2[i] = (-f[i - 2] + 16 * f[i - 1] - 30 * f[i] + 16 * f[i + 1] - f[i + 2]) / (12 * dx * dx);
    }

    // Enforce boundary conditions
    d2f_dx2[0] = d2f_dx2[1] = d2f_dx2[Nx - 2] = d2f_dx2[Nx - 1] = 0.0;
}

// Perform one RK45 step to update f and v
void rk45_step(double *f, double *v, double *d2f_dx2, double dt, double *f_next, double *v_next) {
    double f_stage[Nx], d2f_stage[Nx];
    double kf1[Nx], kv1[Nx], kf2[Nx], kv2[Nx], kf3[Nx], kv3[Nx];
    double kf4[Nx], kv4[Nx], kf5[Nx], kv5[Nx], kf6[Nx], kv6[Nx];

    for (int i = 0; i < Nx; i++) {
        kf1[i] = dt * v[i];
        kv1[i] = dt * c * c * d2f_dx2[i];
    }

    for (int i = 0; i < Nx; i++) f_stage[i] = f[i] + 0.25 * kf1[i];
    compute_spatial_derivatives(f_stage, d2f_stage);
    for (int i = 0; i < Nx; i++) {
        kf2[i] = dt * (v[i] + 0.25 * kv1[i]);
        kv2[i] = dt * c * c * d2f_stage[i];
    }

    for (int i = 0; i < Nx; i++)
        f_stage[i] = f[i] + (3.0/32.0)*kf1[i] + (9.0/32.0)*kf2[i];
    compute_spatial_derivatives(f_stage, d2f_stage);
    for (int i = 0; i < Nx; i++) {
        kf3[i] = dt * (v[i] + (3.0/32.0)*kv1[i] + (9.0/32.0)*kv2[i]);
        kv3[i] = dt * c * c * d2f_stage[i];
    }

    for (int i = 0; i < Nx; i++)
        f_stage[i] = f[i] + (1932.0/2197.0)*kf1[i] - (7200.0/2197.0)*kf2[i] + (7296.0/2197.0)*kf3[i];
    compute_spatial_derivatives(f_stage, d2f_stage);
    for (int i = 0; i < Nx; i++) {
        kf4[i] = dt * (v[i] + (1932.0/2197.0)*kv1[i] - (7200.0/2197.0)*kv2[i] + (7296.0/2197.0)*kv3[i]);
        kv4[i] = dt * c * c * d2f_stage[i];
    }

    for (int i = 0; i < Nx; i++)
        f_stage[i] = f[i] + (439.0/216.0)*kf1[i] - 8.0*kf2[i] + (3680.0/513.0)*kf3[i] - (845.0/4104.0)*kf4[i];
    compute_spatial_derivatives(f_stage, d2f_stage);
    for (int i = 0; i < Nx; i++) {
        kf5[i] = dt * (v[i] + (439.0/216.0)*kv1[i] - 8.0*kv2[i] + (3680.0/513.0)*kv3[i] - (845.0/4104.0)*kv4[i]);
        kv5[i] = dt * c * c * d2f_stage[i];
    }

    for (int i = 0; i < Nx; i++)
        f_stage[i] = f[i] - (8.0/27.0)*kf1[i] + 2.0*kf2[i] - (3544.0/2565.0)*kf3[i]
                     + (1859.0/4104.0)*kf4[i] - (11.0/40.0)*kf5[i];
    compute_spatial_derivatives(f_stage, d2f_stage);
    for (int i = 0; i < Nx; i++) {
        kf6[i] = dt * (v[i] - (8.0/27.0)*kv1[i] + 2.0*kv2[i] - (3544.0/2565.0)*kv3[i]
                        + (1859.0/4104.0)*kv4[i] - (11.0/40.0)*kv5[i]);
        kv6[i] = dt * c * c * d2f_stage[i];
    }

    for (int i = 0; i < Nx; i++) {
        f_next[i] = f[i] + (16.0/135.0)*kf1[i] + (6656.0/12825.0)*kf3[i]
                          + (28561.0/56430.0)*kf4[i] - (9.0/50.0)*kf5[i] + (2.0/55.0)*kf6[i];

        v_next[i] = v[i] + (16.0/135.0)*kv1[i] + (6656.0/12825.0)*kv3[i]
                          + (28561.0/56430.0)*kv4[i] - (9.0/50.0)*kv5[i] + (2.0/55.0)*kv6[i];
    }

    // Enforce Dirichlet boundary conditions
    f_next[0] = f_next[Nx - 1] = 0.0;
    v_next[0] = v_next[Nx - 1] = 0.0;
}

// Save snapshot to file
void save_snapshot(double *f, int timestep) {
    char filename[200];
    snprintf(filename, sizeof(filename), "Finite_Diff_Snapshots/wave_timestep_%d.txt", timestep);

    FILE* file = fopen(filename, "w");
    if (!file) {
        perror("Error opening file");
        exit(EXIT_FAILURE);
    }

    for (int i = 0; i < Nx; i++) {
        fprintf(file, "%f\n", f[i]);
    }

    fclose(file);
}

int main() {
    
    // Arrays for spatial grid and wave amplitudes
    double x[Nx];          // Spatial grid points
    double f[Nx];          // Current wave amplitude
    double v[Nx];          // Current velocity
    double f_next[Nx];     // Next wave amplitude
    double v_next[Nx];     // Next velocity
    double d2f_dx2[Nx];    // Second spatial derivative

    // Create the directory for saving snapshots (ignore if it already exists)
    if (mkdir("Finite_Diff_Snapshots") != 0 && errno != EEXIST) {
        perror("mkdir failed");
        exit(EXIT_FAILURE);
    }

    // Initialize the wave and spatial grid
    initialize_wave(f, v, x);

    // Time propagation loop
    double dt = 0.0005;     // Time step size
    int Nt = 4000;          // Number of time steps

    for (int n = 0; n < Nt; n++) {
        compute_spatial_derivatives(f, d2f_dx2);      // Compute spatial derivatives
        rk45_step(f, v, d2f_dx2, dt, f_next, v_next); // RK45 time integration

        if (n % 20 == 0) {
        save_snapshot(f_next, n);
    }

        // Update arrays for the next iteration
        for (int i = 0; i < Nx; i++) {
            f[i] = f_next[i];
            v[i] = v_next[i];
        }
    }
    return 0;
}

Overwriting finitediff.c


In [55]:
%%cmd
gcc finitediff.c -o finitediff

finitediff.exe

Microsoft Windows [Version 10.0.19045.5737]
(c) Microsoft Corporation. All rights reserved.

c:\Users\dscot\Desktop\University Documents and Projects\2024-2025\PHYS 581\Assignment 5>gcc finitediff.c -o finitediff

c:\Users\dscot\Desktop\University Documents and Projects\2024-2025\PHYS 581\Assignment 5>
c:\Users\dscot\Desktop\University Documents and Projects\2024-2025\PHYS 581\Assignment 5>finitediff.exe

c:\Users\dscot\Desktop\University Documents and Projects\2024-2025\PHYS 581\Assignment 5>

In [56]:
%%file fourier.c

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <sys/stat.h>
#include <errno.h>

#ifdef _WIN32
#include <direct.h>
#define mkdir_if_needed(dir) _mkdir(dir)
#else
#define mkdir_if_needed(dir) mkdir(dir, 0777)
#endif

#define L 1.0
#define a 0.1
#define c 1.0
#define Nx 64
#define PI 3.141592653589793
#define dx (L / (Nx + 1))

// Function prototypes
void initialize_wave(double *f, double *v);
void dst(double *in, double *out);
void idst(double *in, double *out);
void compute_d2f_dx2_sine(double *f, double *d2f_dx2);
void rk45_step(double *f, double *v, double *d2f_dx2, double dt, double *f_next, double *v_next);
void save_snapshot(double *f, int timestep);

int main() {
    double f[Nx], v[Nx], f_next[Nx], v_next[Nx], d2f_dx2[Nx];

    if (mkdir_if_needed("SineFourier_Snapshots") != 0 && errno != EEXIST) {
        perror("mkdir failed");
        exit(EXIT_FAILURE);
    }

    initialize_wave(f, v);

    double dt = 0.0005;
    int Nt = 4000;

    for (int n = 0; n < Nt; n++) {
        compute_d2f_dx2_sine(f, d2f_dx2);
        rk45_step(f, v, d2f_dx2, dt, f_next, v_next);

        if (n % 20 == 0)
            save_snapshot(f_next, n);

        for (int i = 0; i < Nx; i++) {
            f[i] = f_next[i];
            v[i] = v_next[i];
        }
    }

    return 0;
}

// Gaussian initial condition, 0 boundaries implied
void initialize_wave(double *f, double *v) {
    for (int i = 0; i < Nx; i++) {
        double x = -L/2 + (i + 1) * dx;
        f[i] = exp(-x * x / (a * a));
        v[i] = 0.0;
    }
}

// Naive DST-I
void dst(double *in, double *out) {
    for (int k = 0; k < Nx; k++) {
        out[k] = 0.0;
        for (int j = 0; j < Nx; j++) {
            out[k] += in[j] * sin(PI * (j + 1) * (k + 1) / (Nx + 1));
        }
    }
}

// Inverse DST-I (same as forward, scaled)
void idst(double *in, double *out) {
    for (int j = 0; j < Nx; j++) {
        out[j] = 0.0;
        for (int k = 0; k < Nx; k++) {
            out[j] += in[k] * sin(PI * (j + 1) * (k + 1) / (Nx + 1));
        }
        out[j] *= 2.0 / (Nx + 1);
    }
}

void compute_d2f_dx2_sine(double *f, double *d2f_dx2) {
    double f_hat[Nx], d2f_hat[Nx];
    dst(f, f_hat);

    for (int k = 0; k < Nx; k++) {
        double lambda_k = PI * (k + 1) / L;
        d2f_hat[k] = -lambda_k * lambda_k * f_hat[k];
    }

    idst(d2f_hat, d2f_dx2);
}

void rk45_step(double *f, double *v, double *d2f_dx2, double dt, double *f_next, double *v_next) {
    double f_stage[Nx], d2f_stage[Nx];
    double kf1[Nx], kv1[Nx], kf2[Nx], kv2[Nx], kf3[Nx], kv3[Nx];
    double kf4[Nx], kv4[Nx], kf5[Nx], kv5[Nx], kf6[Nx], kv6[Nx];

    for (int i = 0; i < Nx; i++) {
        kf1[i] = dt * v[i];
        kv1[i] = dt * c * c * d2f_dx2[i];
    }

    for (int i = 0; i < Nx; i++) f_stage[i] = f[i] + 0.25 * kf1[i];
    compute_d2f_dx2_sine(f_stage, d2f_stage);
    for (int i = 0; i < Nx; i++) {
        kf2[i] = dt * (v[i] + 0.25 * kv1[i]);
        kv2[i] = dt * c * c * d2f_stage[i];
    }

    for (int i = 0; i < Nx; i++)
        f_stage[i] = f[i] + (3.0/32.0)*kf1[i] + (9.0/32.0)*kf2[i];
    compute_d2f_dx2_sine(f_stage, d2f_stage);
    for (int i = 0; i < Nx; i++) {
        kf3[i] = dt * (v[i] + (3.0/32.0)*kv1[i] + (9.0/32.0)*kv2[i]);
        kv3[i] = dt * c * c * d2f_stage[i];
    }

    for (int i = 0; i < Nx; i++)
        f_stage[i] = f[i] + (1932.0/2197.0)*kf1[i] - (7200.0/2197.0)*kf2[i] + (7296.0/2197.0)*kf3[i];
    compute_d2f_dx2_sine(f_stage, d2f_stage);
    for (int i = 0; i < Nx; i++) {
        kf4[i] = dt * (v[i] + (1932.0/2197.0)*kv1[i] - (7200.0/2197.0)*kv2[i] + (7296.0/2197.0)*kv3[i]);
        kv4[i] = dt * c * c * d2f_stage[i];
    }

    for (int i = 0; i < Nx; i++)
        f_stage[i] = f[i] + (439.0/216.0)*kf1[i] - 8.0*kf2[i] + (3680.0/513.0)*kf3[i] - (845.0/4104.0)*kf4[i];
    compute_d2f_dx2_sine(f_stage, d2f_stage);
    for (int i = 0; i < Nx; i++) {
        kf5[i] = dt * (v[i] + (439.0/216.0)*kv1[i] - 8.0*kv2[i] + (3680.0/513.0)*kv3[i] - (845.0/4104.0)*kv4[i]);
        kv5[i] = dt * c * c * d2f_stage[i];
    }

    for (int i = 0; i < Nx; i++)
        f_stage[i] = f[i] - (8.0/27.0)*kf1[i] + 2.0*kf2[i] - (3544.0/2565.0)*kf3[i]
                     + (1859.0/4104.0)*kf4[i] - (11.0/40.0)*kf5[i];
    compute_d2f_dx2_sine(f_stage, d2f_stage);
    for (int i = 0; i < Nx; i++) {
        kf6[i] = dt * (v[i] - (8.0/27.0)*kv1[i] + 2.0*kv2[i] - (3544.0/2565.0)*kv3[i]
                        + (1859.0/4104.0)*kv4[i] - (11.0/40.0)*kv5[i]);
        kv6[i] = dt * c * c * d2f_stage[i];
    }

    for (int i = 0; i < Nx; i++) {
        f_next[i] = f[i] + (16.0/135.0)*kf1[i] + (6656.0/12825.0)*kf3[i]
                          + (28561.0/56430.0)*kf4[i] - (9.0/50.0)*kf5[i] + (2.0/55.0)*kf6[i];

        v_next[i] = v[i] + (16.0/135.0)*kv1[i] + (6656.0/12825.0)*kv3[i]
                          + (28561.0/56430.0)*kv4[i] - (9.0/50.0)*kv5[i] + (2.0/55.0)*kv6[i];
    }
}

void save_snapshot(double *f, int timestep) {
    char filename[200];
    snprintf(filename, sizeof(filename), "SineFourier_Snapshots/wave_timestep_%d.txt", timestep);
    FILE *file = fopen(filename, "w");
    if (!file) {
        perror("Error writing snapshot");
        exit(EXIT_FAILURE);
    }
    for (int i = 0; i < Nx; i++)
        fprintf(file, "%f\n", f[i]);
    fclose(file);
}


Overwriting fourier.c


In [57]:
%%cmd
gcc fourier.c -o fourier

fourier.exe

Microsoft Windows [Version 10.0.19045.5737]
(c) Microsoft Corporation. All rights reserved.

c:\Users\dscot\Desktop\University Documents and Projects\2024-2025\PHYS 581\Assignment 5>gcc fourier.c -o fourier

c:\Users\dscot\Desktop\University Documents and Projects\2024-2025\PHYS 581\Assignment 5>
c:\Users\dscot\Desktop\University Documents and Projects\2024-2025\PHYS 581\Assignment 5>fourier.exe

c:\Users\dscot\Desktop\University Documents and Projects\2024-2025\PHYS 581\Assignment 5>

In [58]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import time
from matplotlib.animation import PillowWriter
from natsort import natsorted

# Set global domain length for all cases
L = 1.0

def make_animation(snapshot_dir, output_filename, title):
    files = glob.glob(os.path.join(snapshot_dir, "wave_timestep_*.txt"))
    
    if not files:
        print(f"No snapshot files found in {snapshot_dir}")
        return
    files = natsorted(files)

    # Load sample to get spatial grid
    y_sample = np.loadtxt(files[0])
    Nx = len(y_sample)
    x = np.linspace(-L/2, L/2, Nx)
    wave_frames = [np.loadtxt(f) for f in files]

    fig, ax = plt.subplots()
    line, = ax.plot(x, wave_frames[0])
    ax.set_xlim(-L/2, L/2)
    ax.set_ylim(-1.1 * np.max(np.abs(wave_frames)), 1.1 * np.max(np.abs(wave_frames)))
    ax.set_xlabel("x")
    ax.set_ylabel("Amplitude")
    ax.set_title(title)
    ax.grid(True)

    writer = PillowWriter(fps=20)
    print(f"Rendering frames for {output_filename}...")
    start_time = time.time()

    with writer.saving(fig, output_filename, dpi=100):
        total = len(wave_frames)
        for i, frame in enumerate(wave_frames):
            line.set_ydata(frame)
            writer.grab_frame()
            if (i + 1) % (total // 10) == 0:
                print(f"{int((i + 1) / total * 100)}% complete (frame rendering)")

    print("Rendering complete. Finalizing and saving the animation...")
    print(f"Successfully saved: {output_filename}")
    print(f"Total time elapsed: {time.time() - start_time:.2f} seconds\n")
    plt.close(fig)

# Generate animations
make_animation("Finite_Diff_Snapshots", "wave_animation_fd.gif", "Wave Evolution\n(Finite Difference Method)")
make_animation("SineFourier_Snapshots", "wave_animation_fourier.gif", "Wave Evolution\n(Fourier Series Method)")

Rendering frames for wave_animation_fd.gif...
10% complete (frame rendering)
20% complete (frame rendering)
30% complete (frame rendering)
40% complete (frame rendering)
50% complete (frame rendering)
60% complete (frame rendering)
70% complete (frame rendering)
80% complete (frame rendering)
90% complete (frame rendering)
100% complete (frame rendering)
Rendering complete. Finalizing and saving the animation...
Successfully saved: wave_animation_fd.gif
Total time elapsed: 15.18 seconds

Rendering frames for wave_animation_fourier.gif...
10% complete (frame rendering)
20% complete (frame rendering)
30% complete (frame rendering)
40% complete (frame rendering)
50% complete (frame rendering)
60% complete (frame rendering)
70% complete (frame rendering)
80% complete (frame rendering)
90% complete (frame rendering)
100% complete (frame rendering)
Rendering complete. Finalizing and saving the animation...
Successfully saved: wave_animation_fourier.gif
Total time elapsed: 16.75 seconds



In [59]:
code = """\
import os
import glob
import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import PillowWriter
from natsort import natsorted

# Set global domain length for all cases
L = 1.0

def make_animation(snapshot_dir, output_filename, title):
    files = glob.glob(os.path.join(snapshot_dir, "wave_timestep_*.txt"))
    
    if not files:
        print(f"No snapshot files found in {snapshot_dir}")
        return
    files = natsorted(files)

    # Load sample to get spatial grid
    y_sample = np.loadtxt(files[0])
    Nx = len(y_sample)
    x = np.linspace(-L/2, L/2, Nx)
    wave_frames = [np.loadtxt(f) for f in files]

    fig, ax = plt.subplots()
    line, = ax.plot(x, wave_frames[0])
    ax.set_xlim(-L/2, L/2)
    ax.set_ylim(-1.1 * np.max(np.abs(wave_frames)), 1.1 * np.max(np.abs(wave_frames)))
    ax.set_xlabel("x")
    ax.set_ylabel("Amplitude")
    ax.set_title(title)
    ax.grid(True)

    writer = PillowWriter(fps=20)
    print(f"Rendering frames for {output_filename}...")
    start_time = time.time()

    with writer.saving(fig, output_filename, dpi=100):
        total = len(wave_frames)
        for i, frame in enumerate(wave_frames):
            line.set_ydata(frame)
            writer.grab_frame()
            if (i + 1) % (total // 10) == 0:
                print(f"{int((i + 1) / total * 100)}% complete (frame rendering)")

    print("Rendering complete. Finalizing and saving the animation...")
    print(f"Successfully saved: {output_filename}")
    print(f"Total time elapsed: {time.time() - start_time:.2f} seconds\\n")
    plt.close(fig)

# Generate animations
make_animation("Finite_Diff_Snapshots", "wave_animation_fd.gif", "Wave Evolution\\n(Finite Difference Method)")
make_animation("SineFourier_Snapshots", "wave_animation_fourier.gif", "Wave Evolution\\n(Fourier Series Method)")
"""

with open("wave_animation.py", "w", encoding="utf-8", newline="\n") as f:
    f.write(code)

print('Successfully saved as "wave_animation.py".')

Successfully saved as "wave_animation.py".
